In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings(action = 'ignore')
import joblib

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF 벡터화
from sklearn.linear_model import LogisticRegression # 이진분류 알고리즘
from sklearn.model_selection import GridSearchCV # 하이퍼 파라미터 최적화?
from konlpy.tag import Okt # 형태소 분석기
import time

In [3]:
start_time = time.time()
okt = Okt()

In [4]:
# Read excel
train_df = pd.read_excel('5movies.xlsx')

In [5]:
# 결측값 처리
train_df = train_df[train_df['text'].notnull()]

train_df['text'] = train_df['text'].apply(lambda x:
                                         re.sub(r'[^ ㄱ-ㅣ 가-힣] + ', " ", x))

In [6]:
text = train_df['text']
score = train_df['score']

In [7]:
# Train / Test 분리
train_x, test_x, train_y, test_y = train_test_split(text, score , 
                                                    test_size=0.9, random_state=0)

In [8]:
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)

In [9]:
clf = LogisticRegression(random_state=0)
params = {'C': [15, 18, 19, 20, 22]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [15, 18, 19, 20, 22]}, scoring='accuracy',
             verbose=1)

In [10]:
tfv_test_x = tfv.transform(test_x)
# test_predict = grid_cv.best_estimator_.score(tfv_test_x,test_y)
test_predict = grid_cv.best_estimator_.predict(tfv_test_x)

In [11]:
test_input = pd.read_excel('44.xlsx')

In [12]:
test_df = test_input[test_input['Comment'].notnull()]
test_df['Comment'] = test_df['Comment'].apply(lambda x: 
                                        re.sub(r'[^ ㄱ-ㅣ 가-힣] + ', " ", x))

In [13]:
text_r = test_df['Comment']
test = tfv.transform(text_r)

In [14]:
ts_predict = grid_cv.best_estimator_.predict(test)

In [15]:
react_df = pd.DataFrame(columns=['idx', 'react'])

In [16]:
for i in range(len(ts_predict)):
    if(ts_predict[i] == 0):
        react = '부정'
    else:
        react = '긍정'
    
    react_df = react_df.append(pd.DataFrame([[i, react]], columns=['idx', 'react']),
                   ignore_index=True)

# end_time = time.time()
# print(end_time - start_time)

In [17]:
read_df = pd.read_excel('44.xlsx')

In [18]:
read_df = read_df.merge(react_df, how = 'left', 
                        left_on = 'Idx', right_on = 'idx')

In [19]:
columns = ['ID', 'Comment', 'react']

In [20]:
df = read_df[columns]

In [21]:
df

,ID,Comment,react
0,삼성,"""내 손과 다리가 부러져도 그 고통은 이길 수 있사오나, 나라를 잃어버린 그 고통만...",긍정
1,ksns ssj,"지금 팬분들이 해야할건돌아와, 기다릴게, 힘내이런 말이 아니라형은 분명히 잘못했고 ...",긍정
2,도웨이,"""내 손톱이 빠져나가고 내 귀와 코가 잘리고 손과 다리가 부러져도그 고통은 이길 수...",긍정
3,뿌슝빠슝,식민지 역사를 가진 나라에서 그 따위 언행을 하고 대중 상대로 장사하는 BJ가 활동...,긍정
4,L IM,평소에 무슨 생각을 하시는지 알 수 있는 계기가 되었습니다. 사람은 바뀌지 않습니다.,긍정
...,...,...,...
2575,이진행,멀리안간다 잘가라..잘가라는무슨 ㄲㅈ,부정
2576,ty k,이듣보는 누구냐 ㅋㅋㅋ,긍정
2577,류다,니가 사람이냐,긍정
2578,S ong,끄지라,긍정


In [22]:
df.to_excel('total.xlsx')

In [23]:
positive = df[df['react'] == '긍정']

In [24]:
positive

,ID,Comment,react
0,삼성,"""내 손과 다리가 부러져도 그 고통은 이길 수 있사오나, 나라를 잃어버린 그 고통만...",긍정
1,ksns ssj,"지금 팬분들이 해야할건돌아와, 기다릴게, 힘내이런 말이 아니라형은 분명히 잘못했고 ...",긍정
2,도웨이,"""내 손톱이 빠져나가고 내 귀와 코가 잘리고 손과 다리가 부러져도그 고통은 이길 수...",긍정
3,뿌슝빠슝,식민지 역사를 가진 나라에서 그 따위 언행을 하고 대중 상대로 장사하는 BJ가 활동...,긍정
4,L IM,평소에 무슨 생각을 하시는지 알 수 있는 계기가 되었습니다. 사람은 바뀌지 않습니다.,긍정
...,...,...,...
2573,굳굳굳,역겹습니다 유투브 자체를 없애주세요 제발 당신이야기 여기저기 나오는거 정말 역겹습니다,긍정
2576,ty k,이듣보는 누구냐 ㅋㅋㅋ,긍정
2577,류다,니가 사람이냐,긍정
2578,S ong,끄지라,긍정


In [25]:
negative = df[df['react'] == '부정']

In [26]:
negative

,ID,Comment,react
20,민이쪼꼬,손톱이 빠져나가고\n내 귀와 코가 잘리고\n내손과 다리가 부러져도\n그 고통은 이길...,부정
26,김성민,장안동 시절 부터 팬이 였습니다 너무 속상하고 안타깝지만... 그 만큼 정말 많이...,부정
28,오하늘,와 어제까지만 해도 유튭 올린거 재밌게 봤는데 이게 뭔일이냐 많은 사람들이 정 떨어...,부정
33,하마쓰,하 진짜 팬인데.. 형 반성하라는 말밖에 못하겠다,부정
37,울트라,공장길만 걷자 메킴아,부정
...,...,...,...
2569,YoungBin,생각이라는걸 할 줄 모르나?? 어처구니가 없어서 할말을 잃게만드네;;; 진짜 수준 ...,부정
2570,명우김,배가불렀지?,부정
2571,신동준,대한민국에서 나가라 그냥,부정
2574,Jerry,누군데 왈왈하는거임? 짭bj 임? 풉,부정


In [27]:
positive.to_excel('positive.xlsx')

In [28]:
negative.to_excel('negative.xlsx')

In [30]:
positive[['Comment']]

,Comment
0,"""내 손과 다리가 부러져도 그 고통은 이길 수 있사오나, 나라를 잃어버린 그 고통만..."
1,"지금 팬분들이 해야할건돌아와, 기다릴게, 힘내이런 말이 아니라형은 분명히 잘못했고 ..."
2,"""내 손톱이 빠져나가고 내 귀와 코가 잘리고 손과 다리가 부러져도그 고통은 이길 수..."
3,식민지 역사를 가진 나라에서 그 따위 언행을 하고 대중 상대로 장사하는 BJ가 활동...
4,평소에 무슨 생각을 하시는지 알 수 있는 계기가 되었습니다. 사람은 바뀌지 않습니다.
...,...
2573,역겹습니다 유투브 자체를 없애주세요 제발 당신이야기 여기저기 나오는거 정말 역겹습니다
2576,이듣보는 누구냐 ㅋㅋㅋ
2577,니가 사람이냐
2578,끄지라


In [41]:
positive[['Comment']].to_csv('comment.txt', index = False, header = False)